**Preamble:**
There is too much hetrogeniety in knowledge graph embedding (KGE) models, in terms of parameters, perfomance metrics, datasets. It makes comparision of the results/performace of different models difficult. 


**Objectives:**

1.   To reproduce the results of following KGE models with experimental setup given in publication. 

>>> a. NodePiece

> >>b. CompGCN

> >>c. TorusE

> >>d. PairRE

2.   To obtain the results for those KGE models on uniform experimental setup. So that it will be easier later to compare the results. 




# Issue faced

Even after ~250-300hrs of experimentation with various set of parameter values, obtained results are far away than the published results. 

e.g.

CompGCN publication has claimed the hit@10 as 0.5 however I got 0.00158. 


This is the case with all the models, parameter settings, datasets, I have tried. The results obtained for all performance metric are having the same issues. 


I am not able to understand what i am missing here?



In this experimentation we have used the pykeen library. I started with the experimental setup given in its publication. Following code shows the latest trial with different setup.  

In [ ]:
pip install pykeen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# CompGCN



In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    n_trials=1,
   dataset="WN18",
    dataset_kwargs=dict(
        create_inverse_triples=True,
    ),
    model='CompGCN',
    model_kwargs=dict(embedding_dim=64),
    optimizer='SGD',
    optimizer_kwargs=dict(lr=0.0001),
    loss='crossentropy',
    training_loop='slcwa',
    training_kwargs=dict(num_epochs=100, batch_size=512,checkpoint_name='my_checkpoint_CompGCN_1303-5.pt',
    checkpoint_frequency=0,
    checkpoint_directory='/content/drive/MyDrive/Colab Notebooks/checkpoint_dir/'
   ),
    negative_sampler='basic',
    negative_sampler_kwargs=dict(num_negs_per_pos=1),
    evaluator_kwargs=dict(filtered=True),
    evaluation_kwargs=dict(batch_size=128),
    stopper='early',
    stopper_kwargs=dict(frequency=5, patience=2, relative_delta=0.0002),
)
hpo_pipeline_result.save_to_directory('/content/drive/MyDrive/Colab Notebooks/result/Comp_GCN_WN_1303-5')

# NodePiece

In [ ]:
import torch.nn

from pykeen.models import NodePiece
from pykeen.pipeline import pipeline
from pykeen.hpo import hpo_pipeline
from helper import *
result = hpo_pipeline(
     n_trials=1,
    dataset="WN18",
    dataset_kwargs=dict(
        create_inverse_triples=True,
    ),
    model=NodePiece,
    model_kwargs=dict(
        tokenizers=["AnchorTokenizer", "RelationTokenizer"],
        num_tokens=[50, 4],
        tokenizers_kwargs=[
            dict(
                selection="MixtureAnchorSelection",
                selection_kwargs=dict(
                    selections=["degree", "pagerank", "random"],
                    ratios=[0.4, 0.4, 0.2],
                    num_anchors=500,
                ),
                searcher="ScipySparse",
            ),
            dict(),  # empty dict for the RelationTokenizer - it doesn't need any kwargs
        ],
        embedding_dim=2000,
        interaction="rotate",
    ),
    optimizer='adam',
    optimizer_kwargs=dict(lr=0.0001),
    loss='crossentropy',
     training_loop='slcwa',
     training_kwargs=dict(num_epochs=100, batch_size=512,checkpoint_name='my_checkpoint_NodePiece_WN18_1303_3.pt',
    checkpoint_frequency=5,
    checkpoint_directory='/content/drive/MyDrive/Colab Notebooks/checkpoint_dir/'
   ),
)
result.save_to_directory('/content/drive/MyDrive/Colab Notebooks/result/nodepeice_WN18_1303_3')

[I 2023-03-13 08:40:24,516] A new study created in memory with name: no-name-2cd5c4e7-5c6a-4654-90f0-83d0402ccaca
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.node_piece.NodePiece'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.CrossEntropyLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:=> no training loop checkpoint file found at '/content/drive/MyDrive/Colab Notebooks/checkpoint_dir/my_checkpoint_NodePiece_WN18_1303_3.pt'. Creating a new file.
INFO:pykee

sampling:   0%|          | 0.00/40.9k [00:00<?, ?it/s]

INFO:pykeen.training.training_loop:=> no checkpoint found at '/content/drive/MyDrive/Colab Notebooks/checkpoint_dir/my_checkpoint_NodePiece_WN18_1303_3.pt'. Creating a new file.
INFO:pykeen.triples.triples_factory:Creating inverse triples.
INFO:pykeen.training.training_loop:Starting sub_batch_size search for training now...
INFO:pykeen.triples.triples_factory:Creating inverse triples.
INFO:pykeen.triples.triples_factory:Creating inverse triples.
INFO:pykeen.training.training_loop:Concluded search with sub_batch_size 128.


Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

INFO:pykeen.triples.triples_factory:Creating inverse triples.


...

Training batches on cuda:0:   0%|          | 0/553 [00:00<?, ?batch/s]

INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 2.


Training batches on cuda:0:   0%|          | 0/553 [00:00<?, ?batch/s]

INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 3.


Training batches on cuda:0:   0%|          | 0/553 [00:00<?, ?batch/s]

# PairRE

In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    n_trials=10,
    dataset='fb15k237',
    model='pairre',
    loss='MarginRankingLoss',
    model_kwargs_ranges=dict(
         embedding_dim=dict(type=int, low=500, high=500, q=100),
     ),
     training_kwargs=dict(num_epochs=30, batch_size=512,checkpoint_name='my_checkpoint_Pairre_T11.pt',
    checkpoint_frequency=4,   
    checkpoint_directory='/content/drive/MyDrive/Colab Notebooks/checkpoint_dir/'
    ),
    
    optimizer='adam',
    optimizer_kwargs=dict(lr=0.0001),
    training_loop='slcwa',
    loss_kwargs_ranges=dict(
        margin=dict(type=float, low=1.0, high=2.0),
    ),
    
  )
#hpo_pipeline_result.plot()
hpo_pipeline_result.save_to_directory('/content/drive/MyDrive/Colab Notebooks/result/pairre_fb15k237_0703_T11')

INFO:pykeen.utils:Using opt_einsum
[I 2023-03-07 14:49:36,096] A new study created in memory with name: no-name-3ec11571-9f41-47d8-b605-229ba6acc553
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.pair_re.PairRE'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:=> no training loop checkpoint file found at '/content/drive/MyDrive/Colab Notebooks/checkpoint_dir/my_checkpoint_Pairre_T11.pt'. Creating a n

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=14505, num_relations=237, create_inverse_triples=False, num_triples=272115, path=Release/train.txt) to file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.datasets.base:Stored training factory to file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=14505, num_relations=237, create_inverse_triples=False, num_triples=20438, path=Release/test.txt) to file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.datasets.base:Stored testing factory to file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=14505, num_relations=237, create_inverse_triples=False, num_triples=17526, path=Release/valid.txt) to file:///ro

Training epochs on cuda:0:   0%|          | 0/30 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/532 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/532 [00:00<?, ?batch/s]

...

INFO:pykeen.evaluation.evaluator:Evaluation took 29.63s seconds
[I 2023-03-07 14:57:09,593] Trial 8 finished with value: 0.19267258048057556 and parameters: {'model.embedding_dim': 500, 'model.p': 2, 'loss.margin': 1.8954665960834478, 'negative_sampler.num_negs_per_pos': 12}. Best is trial 0 with value: 0.20683042705059052.
INFO:pykeen.pipeline.api:loaded random seed 3177305297 from checkpoint.
INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation
INFO:pykeen.pipeline.api:Us

Training epochs on cuda:0: 100%|##########| 30/30 [00:00<?, ?epoch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=128.


Evaluating on cuda:0:   0%|          | 0.00/17.5k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 29.20s seconds
[I 2023-03-07 14:57:42,599] Trial 9 finished with value: 0.20683042705059052 and parameters: {'model.embedding_dim': 500, 'model.p': 1, 'loss.margin': 1.2766264540212637, 'negative_sampler.num_negs_per_pos': 22}. Best is trial 0 with value: 0.20683042705059052.
